In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/kaggle/input/titanic/train.csv")


In [ ]:

data.head()

In [ ]:
data.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1, inplace=True )

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data['Age'].describe()

In [ ]:
data['Age'].fillna(data['Age'].mean(),inplace=True)

In [ ]:

data.isnull().sum()

In [ ]:
l_sex_dummies=pd.get_dummies(data['Sex'],drop_first=True)

In [ ]:
data= pd.concat([data,l_sex_dummies],axis=1)

In [ ]:
data.head()

In [ ]:
data.drop(['Sex'], axis=1, inplace=True )

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sts =StandardScaler()

In [ ]:

feature_scale = ['Age','Fare']
data[feature_scale] = sts.fit_transform(data[feature_scale])

In [ ]:
data.head()

In [ ]:
X=data.drop(['Survived'],axis=1)
y=data['Survived']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
#create param
model_param = {
    'DecisionTreeClassifier':{
        'model':DecisionTreeClassifier(),
        'param':{
            'criterion': ['gini','entropy']
        }
    },
        'KNeighborsClassifier':{
        'model':KNeighborsClassifier(),
        'param':{
            'n_neighbors': [5,10,15,20,25]
        }
    },
        'SVC':{
        'model':SVC(),
        'param':{
            'kernel':['rbf','linear','sigmoid'],
            'C': [0.1, 1, 10, 100]
         
        }
    }
}

In [ ]:
scores =[]
for model_name, mp in model_param.items():
    model_selection = GridSearchCV(estimator=mp['model'],param_grid=mp['param'],cv=5,return_train_score=False)
    model_selection.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score': model_selection.best_score_,
        'best_params': model_selection.best_params_
    })

In [ ]:
df_model_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_model_score

In [ ]:
model_svc = SVC( C= 100,kernel='rbf')

In [ ]:

model_svc.fit(X, y)

In [ ]:
df2 = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:

df2.head()

In [ ]:
df3=df2.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1 )

In [ ]:
df3.isnull().sum()

In [ ]:
df3['Age'].fillna(df3['Age'].mean(),inplace=True)
df3['Fare'].fillna(df3['Fare'].mean(),inplace=True)

In [ ]:
l_sex_dummies=pd.get_dummies(df3['Sex'],drop_first=True)
df3= pd.concat([df3,l_sex_dummies],axis=1)
df3.drop(['Sex'], axis=1, inplace=True )

In [ ]:
df3.head()

In [ ]:
df3[feature_scale] = sts.fit_transform(df3[feature_scale])

In [ ]:
df3.head()

In [ ]:
y_predicted = model_svc.predict(df3)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": df2['PassengerId'],
        "Survived": y_predicted
    })

In [ ]:
submission.to_csv('titanic_submission_v02.csv', index=False)